<h1 align="center">☁️ - Cloudy regions segmentation 👨‍💻🔬</h1>

<h2 align="center">Modelling - 1st iteration</h2>
<p style="text-align:center">
   Thomas Bury, Afonso Alves, Daniel Staudegger<br>
   Allianz<br>
</p>


Idea start with classification and iteratively select more complex model: 
- Standard Classification of 15 classes (Random Forest and Neural Networks)
- 4 different classifier models for each class
- Object Detection?
- Semantic Segmentation for one class
- Semantic Segmentation for multiple classes

**Question** 💭: do a single NN to predict all four label maks? Or do 4 NN to predict each label mask individually, and then aggregate the predctions?

**Important detail that may affect the perfomance of the NN:** training label masks were generated by scientists by drawing **rectangular shaped bounding boxes**. Alternatively, the label mask could just have been the actual clouds masks belonging the cloud type (instead of retangular, the mask would have the exact shape of the clouds, with many empty spaces between each mask portion - "where you can see the sea" areas). This means the mask label we have contains some "empty information area", corresponding to the cloudless part of the retangular bounding box drawn by the scientist. I don't have enough experience to say that this will make the training harder (e.g. higher runtime and/or worst performance), but at least will affect it, because the NN has to figure out that the cloudless pixels in the proximity of cloud pixels may also belong to the true label mask, and that the resulting prediction typically has a retangular shape. **Do you guys agree with this assessment?**

**Comment about the problem nature:** in reality, there are "4x2=8 problems" that the NN needs to address: for each possible cloud type, decide if the cloud type is present in the image (kinda like a "binary problem"), and if so, decide on a cloud mask. Perhaps this predicament can inform the NN arquitecture (like a "binary decision layer" + "masks guess layer") ?? Research more...

**Other comment about the problem nature:** we could think of problem as a **multiclass softmax problem**, with each pixel having 5 possible classes (None, 'Gravel', 'Flower', 'Sugar', 'Fish'). Nonetheless, we have seen that the cloud bboxes drawn by the scientists can overlap (even if from different cloud labels). This would mean that a pixel can have more than one class associated (e.g y = ('Gravel','Fish') - overlap of two different cloud types in this pixel), making it a **multilabel problem**, with a weird restriction: the None class, if present, can't be associated to other classes (a pixel can't be class None and "Gravel" at the same time), at least if we consider None (empty label) a concrete possible class. 

In [3]:
conda activate pyreidolia


Note: you may need to restart the kernel to use updated packages.


In [38]:
import numpy as np 
import pandas as pd
import seaborn as sns
import scicomap as sc
import matplotlib as mpl
import yaml
from pprint import pprint
import cv2
import matplotlib.pyplot as plt
import os

#To get a progress bar for long loops:
from tqdm.notebook import trange, tqdm
from time import sleep

#Machine Learning and Deep Learning Packages
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Custom package for the project, save all the functions into appropriate sub-packages
from pyreidolia.plot import set_my_plt_style, plot_cloud, plot_rnd_cloud, draw_label_only
from pyreidolia.mask import bounding_box, rle_to_mask, get_binary_mask_sum, mask_to_rle
from pyreidolia.img import get_resolution_sharpness

#Import

In [6]:
%load_ext autoreload
%autoreload 1

In [7]:
%aimport pyreidolia

## Load the config file for the paths
To avoid to hardcode the paths in a versioned file, let's create a `paths.yml` which will **not** be versioned. So that the paths are not overwritten when we pull or merge from the GitHub repo. The `paths.yml` should have a structure like:

```yml
# data
data:
  test: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/data/test_images/"
  train: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/data/train_images/"
  docs: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/data/"

# Path to store all notebooks, ideally not versioned
notebooks: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/notebooks/"

# Path to store all outputs (correlations, jsons, excel, etc)
output: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/output/"

# Path to store all python scripts, for versioning
scripts: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/scripts/"

# Path to studies
studies: "C:/Users/xtbury/Documents/Projects/segmentation_cloudy_regions/studies/"

# Path to reports
reports: "C:/Users/xtbury/Documents/Projects/Pyreidolia/reports/"
```

In [8]:
def string_print(df):
    return print(df.to_string().replace('\n', '\n\t'))

In [9]:
# Where is my yaml ? "C:/Users/A00006617/OneDrive - Allianz Global Investors/Documents/Allianz Data Scientist/Project/Pyreidolia/paths.yml"

paths_yml = input("where is the paths.yml config file?")
with open(paths_yml, "r+") as ymlfile:
    path_dic = yaml.load(ymlfile, Loader=yaml.FullLoader)

pprint(path_dic)

where is the paths.yml config file?C:/Users/A00006617/OneDrive - Allianz Global Investors/Documents/Allianz Data Scientist/Project/Pyreidolia/paths.yml
{'data': {'docs': 'C:/Users/A00006617/OneDrive - Allianz Global '
                  'Investors/Documents/Allianz Data '
                  'Scientist/Project/Pyreidolia/data/',
          'test': 'C:/Users/A00006617/OneDrive - Allianz Global '
                  'Investors/Documents/Allianz Data '
                  'Scientist/Project/Pyreidolia/data/test_images/',
          'train': 'C:/Users/A00006617/OneDrive - Allianz Global '
                   'Investors/Documents/Allianz Data '
                   'Scientist/Project/Pyreidolia/data/train_images/'},
 'notebooks': 'C:/Users/A00006617/OneDrive - Allianz Global '
              'Investors/Documents/Allianz Data '
              'Scientist/Project/Pyreidolia/notebooks/',
 'output': 'C:/Users/A00006617/OneDrive - Allianz Global '
           'Investors/Documents/Allianz Data '
           'Scie

In [10]:
train_csv_path = path_dic['data']['docs'] + 'train.csv'
train_pq_path = path_dic['data']['docs'] + "train_info_clean.parquet"
train_data = path_dic['data']['train'] 
test_data = path_dic['data']['test'] 
report_path = path_dic['reports']

## Load pre-processed data

In [29]:
#Load the cleaned training meta-data
train_doc = pd.read_parquet(train_pq_path)
train_doc.head(4)
#and convert to grouped-masks dataframe to obtain one row per image_id
grouped_labels = train_doc.loc[train_doc.is_mask, :].groupby('image_id').agg({'label': lambda x: list(x)})
grouped_labels["label_comb"] = grouped_labels["label"].str.join("-")


In [17]:
#Load X_train2 and X_valid2 - containing rows of arrays with the pixels of the pre-processed images from train_doc
# already split in validation and test set
with open(path_dic['data']['docs'] + 'X_train2.npy', 'rb') as f:
    X_train2 = np.load(f)

with open(path_dic['data']['docs'] + 'X_valid2.npy', 'rb') as f:
    X_valid2 = np.load(f)
    
#we also load the image_ids of train and test
with open(path_dic['data']['docs'] + 'Img_train2.pkl', 'rb') as f:
    Img_train2 = pd.read_pickle(f)

with open(path_dic['data']['docs'] + 'Img_valid2.pkl', 'rb') as f:
    Img_valid2 = pd.read_pickle(f)

In [63]:
#Load y_train2_IC, y_valid2_IC
with open(path_dic['data']['docs'] + 'y_train2_IC.npy', 'rb') as f:
    y_train2_IC = np.load(f)
    
with open(path_dic['data']['docs'] + 'y_valid2_IC.npy', 'rb') as f:
    y_valid2_IC = np.load(f)

255

## 1. Modeling - Classification (15 classes)

We start by simplifying the problem and converting it to a standard classification problem. Since standard algorithms cannot detect multiple objects per image, we treat the problem as a classifcation problem with 15 classes (= number of unique label combinations). \
We then try a Random Forest as well as a CNNs with different architecture for classification.

In [34]:
y_train2_labels = Img_train2.merge(grouped_labels, on = 'image_id', how = 'left')['label_comb']
y_valid2_labels = Img_valid2.merge(grouped_labels, on = 'image_id', how = 'left')['label_comb']
#Encode the 15 classes to 0-15 with Label-Encoder
le = LabelEncoder()
y_train2 = le.fit_transform(y_train2_labels)
y_valid2 = le.transform(y_valid2_labels)

#We see that indeed all label combinations where present and encoded
le.classes_

array(['Fish', 'Fish-Flower', 'Fish-Flower-Gravel',
       'Fish-Flower-Gravel-Sugar', 'Fish-Flower-Sugar', 'Fish-Gravel',
       'Fish-Gravel-Sugar', 'Fish-Sugar', 'Flower', 'Flower-Gravel',
       'Flower-Gravel-Sugar', 'Flower-Sugar', 'Gravel', 'Gravel-Sugar',
       'Sugar'], dtype=object)

### 1.1. Random Forest

In [36]:
#RandomForest model
clf_rf = RandomForestClassifier(n_jobs = -1)
clf_rf.fit(X_train2, y_train2)


RandomForestClassifier(n_jobs=-1)

In [42]:
#Model Evaluation
y_pred_rf = clf_rf.predict(X_valid2)
print(classification_report(y_valid2, y_pred_rf))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        70
           1       0.00      0.00      0.00        75
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00        66
           4       0.00      0.00      0.00        77
           5       0.00      0.00      0.00        62
           6       0.13      0.37      0.19       190
           7       0.10      0.05      0.07       128
           8       0.07      0.01      0.02        82
           9       0.00      0.00      0.00        38
          10       0.06      0.01      0.02       104
          11       0.10      0.05      0.07       111
          12       0.00      0.00      0.00        77
          13       0.13      0.43      0.20       176
          14       0.04      0.02      0.03        86

    accuracy                           0.12      1386
   macro avg       0.04      0.06      0.04      1386
weighted avg       0.06   

C:\Winapp\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Unexpectedly quite a poor performance. Let's see if a Neural Network can do any better

### 1.2. Dense Neural Network

In [46]:
y_train_cat = to_categorical(y_train2, dtype = 'int') # Veiller à n'exécuter cette instruction qu'une seule fois
y_valid_cat = to_categorical(y_valid2, dtype = 'int')   # Veiller à n'exécuter cette instruction qu'une seule fois

num_pixels = X_train2.shape[1]
num_classes = y_train_cat.shape[1]

In [50]:
#Simple Network Architecture
inputs = Input(shape = (num_pixels), name = "Input")
dense1 = Dense(units = 20, activation = "tanh", name = "first_layer")
dense2 = Dense(units = num_classes, activation = "softmax", name = "second_layer")
x = dense1(inputs)
outputs = dense2(x)

model = Model(inputs = inputs, outputs = outputs)

In [52]:
#Model Training
model.compile(loss = "categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])
training_history = model.fit(X_train2, y_train_cat, 
                             epochs = 20, 
                             batch_size = 32, 
                             validation_split = 0.2)

Epoch 1/20
104/104 [==============================] - 4s 39ms/step - loss: 2.7338 - accuracy: 0.1199 - val_loss: 2.6432 - val_accuracy: 0.1166
Epoch 2/20
104/104 [==============================] - 4s 38ms/step - loss: 2.6148 - accuracy: 0.1289 - val_loss: 2.6404 - val_accuracy: 0.1166
Epoch 3/20
104/104 [==============================] - 4s 40ms/step - loss: 2.6116 - accuracy: 0.1328 - val_loss: 2.6423 - val_accuracy: 0.1298
Epoch 4/20
104/104 [==============================] - 4s 39ms/step - loss: 2.6104 - accuracy: 0.1358 - val_loss: 2.6435 - val_accuracy: 0.1298
Epoch 5/20
104/104 [==============================] - 4s 40ms/step - loss: 2.6112 - accuracy: 0.1316 - val_loss: 2.6443 - val_accuracy: 0.1298
Epoch 6/20
104/104 [==============================] - 4s 39ms/step - loss: 2.6107 - accuracy: 0.1361 - val_loss: 2.6427 - val_accuracy: 0.1166
Epoch 7/20
104/104 [==============================] - 4s 39ms/step - loss: 2.6113 - accuracy: 0.1340 - val_loss: 2.6434 - val_accuracy: 0.1166

In [53]:
#Model Evaluation
y_pred_dnn  = model.predict(X_valid2)
y_pred_dnn_class = np.argmax(y_pred_dnn, axis = 1)  #Turn probabilities into vector with the predicted class
print(classification_report(y_valid2, y_pred_dnn_class))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        70
           1       0.00      0.00      0.00        75
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00        66
           4       0.00      0.00      0.00        77
           5       0.00      0.00      0.00        62
           6       0.14      1.00      0.24       190
           7       0.00      0.00      0.00       128
           8       0.00      0.00      0.00        82
           9       0.00      0.00      0.00        38
          10       0.00      0.00      0.00       104
          11       0.00      0.00      0.00       111
          12       0.00      0.00      0.00        77
          13       0.00      0.00      0.00       176
          14       0.00      0.00      0.00        86

    accuracy                           0.14      1386
   macro avg       0.01      0.07      0.02      1386
weighted avg       0.02   

Slightly better, but still poor. 
We note the model was succesful by focusing on only one specific class. Hence we next try to change the approach and see if a classifier can identify the existence of a specific cloud type in an image.

## 2. Modeling - Classification (1 class)

We now move to a different approach, where we create a separate classifier for each label and see if this way we can get a higher accuracy.

In [69]:
#Create the 4 different training and validation vectors - the order of the labels come from 01a_preprocessing notebook
y_train2_IC_fish = y_train2_IC[:,0]
y_train2_IC_flower = y_train2_IC[:,1]
y_train2_IC_gravel = y_train2_IC[:,2]
y_train2_IC_sugar = y_train2_IC[:,3]

y_valid2_IC_fish = y_valid2_IC[:,0]
y_valid2_IC_flower = y_valid2_IC[:,1]
y_valid2_IC_gravel = y_valid2_IC[:,2]
y_valid2_IC_sugar = y_valid2_IC[:,3]

### 2.1. Random Forest

In [76]:
#We create 4 Random Forest Classifiers - one for each label
clf_rf_fish = RandomForestClassifier(n_jobs = -1)
clf_rf_flower = RandomForestClassifier(n_jobs = -1)
clf_rf_gravel = RandomForestClassifier(n_jobs = -1)
clf_rf_sugar = RandomForestClassifier(n_jobs = -1)

clf_rf_fish.fit(X_train2, y_train2_IC_fish)
clf_rf_flower.fit(X_train2, y_train2_IC_flower)
clf_rf_gravel.fit(X_train2, y_train2_IC_gravel)
clf_rf_sugar.fit(X_train2, y_train2_IC_sugar)

RandomForestClassifier(n_jobs=-1)

In [77]:
# Predictions and model evaluation
y_pred_rf_fish = clf_rf_fish.predict(X_valid2)
y_pred_rf_flower = clf_rf_flower.predict(X_valid2)
y_pred_rf_gravel = clf_rf_gravel.predict(X_valid2)
y_pred_rf_sugar = clf_rf_sugar.predict(X_valid2)
print(classification_report(y_valid2_IC_fish, y_pred_rf_fish),
     classification_report(y_valid2_IC_flower, y_pred_rf_flower),
     classification_report(y_valid2_IC_gravel, y_pred_rf_gravel),
     classification_report(y_valid2_IC_sugar, y_pred_rf_sugar))

              precision    recall  f1-score   support

           0       0.47      0.52      0.49       674
           1       0.49      0.44      0.47       712

    accuracy                           0.48      1386
   macro avg       0.48      0.48      0.48      1386
weighted avg       0.48      0.48      0.48      1386
               precision    recall  f1-score   support

           0       0.57      0.88      0.69       789
           1       0.44      0.13      0.20       597

    accuracy                           0.55      1386
   macro avg       0.51      0.50      0.45      1386
weighted avg       0.52      0.55      0.48      1386
               precision    recall  f1-score   support

           0       0.45      0.41      0.42       629
           1       0.54      0.58      0.56       757

    accuracy                           0.50      1386
   macro avg       0.49      0.49      0.49      1386
weighted avg       0.50      0.50      0.50      1386
               preci

This model already gives better results - in particular for Sugar it seems to work not too bad. We remember that Sugar are typically wide-spread cloud structures, so perhaps the classifier can spot large structures of Clouds fairly well. 

### 2.2. Dense Neural Network

In [82]:
num_pixels = X_train2.shape[1]
num_classes = 1

In [83]:
# We start by choosing the same simple Network Architecture as above
inputs = Input(shape = (num_pixels), name = "Input")
dense1 = Dense(units = 20, activation = "relu", name = "first_layer")
dense2 = Dense(units = num_classes, activation = "sigmoid", name = "second_layer")
x = dense1(inputs)
outputs = dense2(x)

model_fish = Model(inputs = inputs, outputs = outputs)
model_flower = Model(inputs = inputs, outputs = outputs)
model_gravel = Model(inputs = inputs, outputs = outputs)
model_sugar = Model(inputs = inputs, outputs = outputs)

In [84]:
model_fish.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])
model_flower.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])
model_gravel.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])
model_sugar.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

training_history_fish = model_fish.fit(X_train2, y_train2_IC_fish, epochs = 20, batch_size = 32, validation_split = 0.2)
training_history_flower = model_flower.fit(X_train2, y_train2_IC_flower, epochs = 20, batch_size = 32, validation_split = 0.2)
training_history_gravel = model_gravel.fit(X_train2, y_train2_IC_gravel, epochs = 20, batch_size = 32, validation_split = 0.2)
training_history_sugar = model_sugar.fit(X_train2, y_train2_IC_sugar, epochs = 20, batch_size = 32, validation_split = 0.2)

Epoch 1/20
104/104 [==============================] - 3s 31ms/step - loss: 1295.3586 - accuracy: 0.4985 - val_loss: 0.6932 - val_accuracy: 0.4904
Epoch 2/20
104/104 [==============================] - 3s 27ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6932 - val_accuracy: 0.4904
Epoch 3/20
104/104 [==============================] - 3s 30ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6933 - val_accuracy: 0.4904
Epoch 4/20
104/104 [==============================] - 3s 29ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6933 - val_accuracy: 0.4904
Epoch 5/20
104/104 [==============================] - 3s 31ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6934 - val_accuracy: 0.4904
Epoch 6/20
104/104 [==============================] - 3s 29ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6933 - val_accuracy: 0.4904
Epoch 7/20
104/104 [==============================] - 3s 26ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6934 - val_accuracy: 0.4

104/104 [==============================] - 3s 28ms/step - loss: 0.6923 - accuracy: 0.5213 - val_loss: 0.6909 - val_accuracy: 0.5373
Epoch 19/20
104/104 [==============================] - 3s 29ms/step - loss: 0.6923 - accuracy: 0.5213 - val_loss: 0.6909 - val_accuracy: 0.5373
Epoch 20/20
104/104 [==============================] - 3s 27ms/step - loss: 0.6923 - accuracy: 0.5213 - val_loss: 0.6909 - val_accuracy: 0.5373
Epoch 1/20
104/104 [==============================] - 4s 33ms/step - loss: 0.6721 - accuracy: 0.6794 - val_loss: 0.6685 - val_accuracy: 0.6635
Epoch 2/20
104/104 [==============================] - 3s 32ms/step - loss: 0.6604 - accuracy: 0.6794 - val_loss: 0.6595 - val_accuracy: 0.6635
Epoch 3/20
104/104 [==============================] - 3s 30ms/step - loss: 0.6512 - accuracy: 0.6794 - val_loss: 0.6527 - val_accuracy: 0.6635
Epoch 4/20
104/104 [==============================] - 3s 30ms/step - loss: 0.6444 - accuracy: 0.6794 - val_loss: 0.6478 - val_accuracy: 0.6635
Epoch 5/

In [101]:
# Prediction and model evaluation
y_pred_dnn_fish = model_fish.predict(X_valid2)
y_pred_dnn_flower = model_flower.predict(X_valid2)
y_pred_dnn_gravel = model_gravel.predict(X_valid2)
y_pred_dnn_sugar = model_sugar.predict(X_valid2)

score_fish = model_fish.evaluate(X_valid2, y_valid2_IC_fish, verbose=0)
score_flower = model_flower.evaluate(X_valid2, y_valid2_IC_flower, verbose=0)
score_gravel = model_gravel.evaluate(X_valid2, y_valid2_IC_gravel, verbose=0)
score_sugar = model_sugar.evaluate(X_valid2, y_valid2_IC_sugar, verbose=0)

print("Fish model score %s: %.2f%%" % (model_fish.metrics_names[1], score_fish[1]*100))
print("Flower model score %s: %.2f%%" % (model_flower.metrics_names[1], score_flower[1]*100))
print("Gravel model score %s: %.2f%%" % (model_gravel.metrics_names[1], score_gravel[1]*100))
print("Sugar model score %s: %.2f%%" % (model_sugar.metrics_names[1], score_sugar[1]*100))


Fish model score accuracy: 51.37%
Flower model score accuracy: 43.07%
Gravel model score accuracy: 54.62%
Sugar model score accuracy: 67.68%


In [102]:
#Looking closer though, we notice that the neural network has built a very simple algorithm that always outputs the same probability
print('Fish - max:', np.max(y_pred_dnn_fish), ', min: ', np.min(y_pred_dnn_fish),
      '\n Flower - max:', np.max(y_pred_dnn_flower), ', min: ', np.min(y_pred_dnn_flower),
      '\n Gravel - max:', np.max(y_pred_dnn_gravel), ', min: ', np.min(y_pred_dnn_gravel),
      '\n Sugar - max:', np.max(y_pred_dnn_sugar), ', min: ', np.min(y_pred_dnn_sugar))


Fish - max: 0.6771052 , min:  0.6771052 
 Flower - max: 0.6771052 , min:  0.6771052 
 Gravel - max: 0.6771052 , min:  0.6771052 
 Sugar - max: 0.6771052 , min:  0.6771052


To sum up, defining one classifier per image definitely gives better results than the initial 15-class classifier. Nonetheless, the DNN model appears to use some "short-cuts" and ignore the information provided by the features. 
\
\
We can of course try to fine-tune the model, e.g. by using a different architecture (LeNet) as well as play around with the hyperparameters (number of layers, activiation functions, etc), as well as combining these 4 classifiers into a single classifier. 
\
Otherwise, we can put the classification models aside and move to the next modeling category where we will reframe the problem into an Object Detection task. 